<a href="https://colab.research.google.com/github/AlexKressner/Industrielles_Management/blob/main/Produktionsplanungsproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Produktionsplanungsproblem

In [ ]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [ ]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Indexmengen

In [ ]:
J = ['P1','P2'] # Menge der Produkte
I = ['Maschinen','Rohstoffe','Montage'] # Menge der Produktionsressourcen

## Entscheidungsvariablen

In [ ]:
infinity = solver.infinity()

In [ ]:
# Definition der Entscheidungsvariablen
# j=1 -> P1 -> x[P1]
# j=2 -> P2 -> x[P2]
x={}
for j in J:
    x[j] = solver.NumVar(0.0, infinity, j)
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

## Parameter

In [ ]:
c={} # Deckungsbeiträge
c['P1']=15
c['P2']=20

In [ ]:
a={} # Produktionskoeffizienten, a[ij]
a['Maschinen']={'P1':1,'P2':1}
a['Rohstoffe']={'P1':6,'P2':9}
a['Montage']={'P1':0,'P2':1}

In [ ]:
b={} # Kapazitäten Produktionsfaktoren
b['Maschinen']=100
b['Rohstoffe']=720
b['Montage']=60

## Zielfunktion

In [ ]:
# Maximierung der gesamten Deckungsbeiträge
solver.Maximize(sum(c[j]*x[j] for j in J))

## Nebenbedingungen

In [ ]:
# i=1=Maschine -> x[P1] + x[P2] <= 100
# i=2=Rohstoffe -> 6x[P1] + 9x[P2] <= 720
# i=3=Endmontage -> x[P2] <= 60
for i in I:
    solver.Add(sum(a[i][j]*x[j] for j in J)<=b[i])
print('Anzahl Nebenbedingungen =', solver.NumConstraints())

## Berechnung Lösung

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', solver.Objective().Value())
    for j in J:
        print(f'{j} =', round(x[j].solution_value()))
else:
    print('Problem hat keine Lösung')